In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


---
# PITCHING

In [31]:
sphome2023_era = pd.read_csv("starting_pitcher_data_home_2023.csv")
spaway2023_era = pd.read_csv("starting_pitcher_data_away_2023.csv")
sphome2023_era.head()

,Unnamed: 0,game_pk,home_team,home_sp,np,ip,er,k,bb,h
0,0,718767,SEA,622491,76,6.00,0.0,6,0,1
1,1,718782,BOS,446372,80,3.33,5.0,4,4,6
2,2,718780,WSH,571578,85,3.00,2.0,3,3,7
3,3,718779,TEX,594798,73,3.67,5.0,7,0,6
4,4,718778,SD,605483,93,4.33,3.0,9,1,6


In [32]:
spaway2023_era = spaway2023_era.rename(columns={'away_team':'team','away_sp':'pitcher','np':'n_pitches'})
spaway2023_era.columns.values[0] = 'game_sequence'
spaway2023_era.insert(3,'is_home',False)
spaway2023_era['game_sequence']+=1
spaway2023_era.head()

,game_sequence,game_pk,team,is_home,pitcher,n_pitches,ip,er,k,bb,h
0,1,718767,CLE,False,669456,87,6.00,0.0,3,0,6
1,2,718782,BAL,False,502043,79,5.00,4.0,3,1,6
2,3,718780,ATL,False,608331,43,3.33,1.0,2,0,4
3,4,718779,PHI,False,605400,72,3.67,5.0,4,2,4
4,5,718778,COL,False,608566,87,6.00,2.0,5,0,5


In [33]:
sphome2023_era = sphome2023_era.rename(columns={'home_team':'team','home_sp':'pitcher','np':'n_pitches'})
sphome2023_era.columns.values[0] = 'game_sequence'
sphome2023_era.insert(3,'is_home',True)
sphome2023_era['game_sequence']+=1
sphome2023_era.head()

,game_sequence,game_pk,team,is_home,pitcher,n_pitches,ip,er,k,bb,h
0,1,718767,SEA,True,622491,76,6.00,0.0,6,0,1
1,2,718782,BOS,True,446372,80,3.33,5.0,4,4,6
2,3,718780,WSH,True,571578,85,3.00,2.0,3,3,7
3,4,718779,TEX,True,594798,73,3.67,5.0,7,0,6
4,5,718778,SD,True,605483,93,4.33,3.0,9,1,6


In [34]:
sp2023_era = pd.concat([sphome2023_era,spaway2023_era],axis=0).sort_values(by=['game_sequence','is_home']).reset_index(drop=True)
sp2023_era.head(10)

,game_sequence,game_pk,team,is_home,pitcher,n_pitches,ip,er,k,bb,h
0,1,718767,CLE,False,669456,87,6.00,0.0,3,0,6
1,1,718767,SEA,True,622491,76,6.00,0.0,6,0,1
2,2,718782,BAL,False,502043,79,5.00,4.0,3,1,6
3,2,718782,BOS,True,446372,80,3.33,5.0,4,4,6
4,3,718780,ATL,False,608331,43,3.33,1.0,2,0,4
5,3,718780,WSH,True,571578,85,3.00,2.0,3,3,7
6,4,718779,PHI,False,605400,72,3.67,5.0,4,2,4
7,4,718779,TEX,True,594798,73,3.67,5.0,7,0,6
8,5,718778,COL,False,608566,87,6.00,2.0,5,0,5
9,5,718778,SD,True,605483,93,4.33,3.0,9,1,6


In [35]:
sp2023_era.to_csv('cleaned_pitching_data_2023_starter_era.csv', index=True)

---
# BATTING

In [15]:
df = pd.read_csv('data_without_metric_2023.csv')
df.head()

,Unnamed: 0,game_pk,home_result,date,away_team,home_team,away_final_score,home_final_score,away_starting_pitcher,home_starting_pitcher,...,away_b9,home_b1,home_b2,home_b3,home_b4,home_b5,home_b6,home_b7,home_b8,home_b9
0,0,718767,W,2023-03-30,CLE,SEA,0,3,669456,622491,...,664702,677594,543939,664034,606192,663728,553993,672284,600303,641487
1,1,718782,L,2023-03-30,BAL,BOS,10,9,502043,446372,...,622761,657077,646240,457759,807799,594807,671213,624414,624512,571771
2,2,718780,L,2023-03-30,ATL,WSH,7,2,608331,571578,...,606115,657041,608841,600869,642086,660688,669743,671277,645302,682928
3,3,718779,W,2023-03-30,PHI,TEX,7,11,605400,594798,...,669016,543760,608369,663993,666969,673962,641680,543257,543543,669701
4,4,718778,L,2023-03-30,COL,SD,7,2,608566,605483,...,678662,663757,665742,592518,593428,630105,572761,543592,673490,621311


In [56]:
game_away = df[['game_pk','away_team']].rename(columns = {'away_team':'team'})
game_home = df[['game_pk','home_team']].rename(columns = {'home_team':'team'})
game_away.insert(0,'game_sequence',game_away.index+1)
game_home.insert(0,'game_sequence',game_home.index+1)
game_away['is_home'] = False
game_home['is_home'] = True
game_away.head()

,game_sequence,game_pk,team,is_home
0,1,718767,CLE,False
1,2,718782,BAL,False
2,3,718780,ATL,False
3,4,718779,PHI,False
4,5,718778,COL,False


In [57]:
game_home.head()

,game_sequence,game_pk,team,is_home
0,1,718767,SEA,True
1,2,718782,BOS,True
2,3,718780,WSH,True
3,4,718779,TEX,True
4,5,718778,SD,True


In [85]:
batting_away_lineup = pd.read_csv('batting_data_away_2023.csv')
batting_home_lineup = pd.read_csv('batting_data_home_2023.csv')

In [86]:
batting_away_lineup.head(10)

,game_pk,batter,ab,bb,hbp,single,double,triple,hr,sf
0,718767,680757,4,0,0,0,0,0,0,0
1,718767,642708,4,0,0,1,0,0,0,0
2,718767,608070,4,0,0,0,1,0,0,0
3,718767,605137,4,0,0,1,0,0,0,0
4,718767,647304,3,0,0,0,0,0,0,0
5,718767,665926,3,0,0,0,0,0,0,0
6,718767,686823,3,0,0,1,0,0,0,0
7,718767,572287,3,0,0,0,0,0,0,0
8,718767,664702,3,0,0,0,0,0,0,0
9,718768,641313,4,1,0,1,1,0,0,0


In [74]:
def clean_batting(df):
    # df.insert(0,'game_sequence',(df.index+9)//9)
    # df.insert(2,'is_home',is_home)
    df.insert(2,'batting_order',df.index%9+1)
    # df = df.groupby(game_pk)
    return df

In [75]:
cleaning_batting_away = clean_batting(batting_away_lineup)
cleaning_batting_away.head(20)

,game_pk,batter,batting_order,ab,bb,hbp,single,double,triple,hr,sf
0,718767,680757,1,4,0,0,0,0,0,0,0
1,718767,642708,2,4,0,0,1,0,0,0,0
2,718767,608070,3,4,0,0,0,1,0,0,0
3,718767,605137,4,4,0,0,1,0,0,0,0
4,718767,647304,5,3,0,0,0,0,0,0,0
5,718767,665926,6,3,0,0,0,0,0,0,0
6,718767,686823,7,3,0,0,1,0,0,0,0
7,718767,572287,8,3,0,0,0,0,0,0,0
8,718767,664702,9,3,0,0,0,0,0,0,0
9,718768,641313,1,4,1,0,1,1,0,0,0


In [76]:
cleaning_batting_home = clean_batting(batting_home_lineup)
cleaning_batting_home.head(20)

,game_pk,batter,batting_order,ab,bb,hbp,single,double,triple,hr,sf
0,718767,677594,1,4,0,0,0,0,0,0,0
1,718767,543939,2,3,0,1,0,0,0,0,0
2,718767,664034,3,4,0,0,1,1,0,1,0
3,718767,606192,4,4,0,0,0,0,0,0,0
4,718767,663728,5,4,0,0,1,0,0,0,0
5,718767,553993,6,3,0,0,0,1,0,0,0
6,718767,672284,7,3,0,0,1,0,0,0,0
7,718767,600303,8,3,0,0,0,0,0,0,0
8,718767,641487,9,2,1,0,1,0,0,0,0
9,718768,665161,1,4,0,0,1,0,0,0,0


In [77]:
cleaning_batting_away_with_team = pd.merge(game_away,cleaning_batting_away,on='game_pk')
cleaning_batting_away_with_team.head(20)

,game_sequence,game_pk,team,is_home,batter,batting_order,ab,bb,hbp,single,double,triple,hr,sf
0,1,718767,CLE,False,680757,1,4,0,0,0,0,0,0,0
1,1,718767,CLE,False,642708,2,4,0,0,1,0,0,0,0
2,1,718767,CLE,False,608070,3,4,0,0,0,1,0,0,0
3,1,718767,CLE,False,605137,4,4,0,0,1,0,0,0,0
4,1,718767,CLE,False,647304,5,3,0,0,0,0,0,0,0
5,1,718767,CLE,False,665926,6,3,0,0,0,0,0,0,0
6,1,718767,CLE,False,686823,7,3,0,0,1,0,0,0,0
7,1,718767,CLE,False,572287,8,3,0,0,0,0,0,0,0
8,1,718767,CLE,False,664702,9,3,0,0,0,0,0,0,0
9,2,718782,BAL,False,656775,1,4,2,0,1,0,0,0,0


In [80]:
cleaning_batting_home_with_team = pd.merge(game_home,cleaning_batting_home,on='game_pk')
cleaning_batting_home_with_team.head(20)

,game_sequence,game_pk,team,is_home,batter,batting_order,ab,bb,hbp,single,double,triple,hr,sf
0,1,718767,SEA,True,677594,1,4,0,0,0,0,0,0,0
1,1,718767,SEA,True,543939,2,3,0,1,0,0,0,0,0
2,1,718767,SEA,True,664034,3,4,0,0,1,1,0,1,0
3,1,718767,SEA,True,606192,4,4,0,0,0,0,0,0,0
4,1,718767,SEA,True,663728,5,4,0,0,1,0,0,0,0
5,1,718767,SEA,True,553993,6,3,0,0,0,1,0,0,0
6,1,718767,SEA,True,672284,7,3,0,0,1,0,0,0,0
7,1,718767,SEA,True,600303,8,3,0,0,0,0,0,0,0
8,1,718767,SEA,True,641487,9,2,1,0,1,0,0,0,0
9,2,718782,BOS,True,657077,1,5,0,0,1,0,1,0,0


In [81]:
batting2023 = pd.concat([cleaning_batting_away_with_team,cleaning_batting_home_with_team],axis=0).sort_values(by=['game_sequence','is_home']).reset_index(drop=True)
batting2023.head(20)

,game_sequence,game_pk,team,is_home,batter,batting_order,ab,bb,hbp,single,double,triple,hr,sf
0,1,718767,CLE,False,680757,1,4,0,0,0,0,0,0,0
1,1,718767,CLE,False,642708,2,4,0,0,1,0,0,0,0
2,1,718767,CLE,False,608070,3,4,0,0,0,1,0,0,0
3,1,718767,CLE,False,605137,4,4,0,0,1,0,0,0,0
4,1,718767,CLE,False,647304,5,3,0,0,0,0,0,0,0
5,1,718767,CLE,False,665926,6,3,0,0,0,0,0,0,0
6,1,718767,CLE,False,686823,7,3,0,0,1,0,0,0,0
7,1,718767,CLE,False,572287,8,3,0,0,0,0,0,0,0
8,1,718767,CLE,False,664702,9,3,0,0,0,0,0,0,0
9,1,718767,SEA,True,677594,1,4,0,0,0,0,0,0,0


In [82]:
batting2023.to_csv('cleaned_batting_data_2023.csv', index=True)

In [42]:
# archive: pivot the table (9 rows into 1 row)

# def pivot_df(df):
#     grouped = df.groupby('game_pk')
#     for i in grouped:
#         ab_pivot = df.set_index('batting_order')['ab']
#         bb_pivot = df.set_index('batting_order')['bb']
#         hbp_pivot = df.set_index('batting_order')['hbp']
#         single_pivot = df.set_index('batting_order')['single']
#         double_pivot = df.set_index('batting_order')['double']
#         triple_pivot = df.set_index('batting_order')['triple']
#         hr_pivot = df.set_index('batting_order')['hr']
#         sf_pivot = df.set_index('batting_order')['sf']
        



